In [1]:
import pandas as pd
import numpy as np

# Traitement du dataset entier 

In [6]:
# load json in dataframe 
df = pd.read_csv("conso-inf36-region.csv", sep=";")

In [7]:
df.head()

,Horodate,Région,Code région,Profil,Plage de puissance souscrite,Nb points soutirage,Total énergie soutirée (Wh),Courbe Moyenne n°1 (Wh),Indice représentativité Courbe n°1 (%),Courbe Moyenne n°2 (Wh),Indice représentativité Courbe n°2 (%),Courbe Moyenne n°1 + n°2 (Wh),Indice représentativité Courbe n°1 + n°2 (%),Jour max du mois (0/1),Semaine max du mois (0/1)
0,2022-09-30T23:30:00+02:00,Auvergne-Rhône-Alpes,84,ENT3 (+ ENT4 + ENT5),P0: Total <= 36 kVA,1376,2095739.0,2084.0,48,1368.0,48,1727.0,97,0,1
1,2022-09-30T23:30:00+02:00,Auvergne-Rhône-Alpes,84,PRO1 (+ PRO1WE),P4: ]9-12] kVA,52691,12376840.0,443.0,8,202.0,8,322.0,16,0,1
2,2022-09-30T23:30:00+02:00,Auvergne-Rhône-Alpes,84,PRO1 (+ PRO1WE),P8: ]24-30] kVA,8350,6545703.0,1012.0,10,474.0,11,742.0,22,0,1
3,2022-09-30T23:30:00+02:00,Auvergne-Rhône-Alpes,84,PRO2 (+ PRO2WE + PRO6),P3: ]6-9] kVA,19759,7072016.0,693.0,13,466.0,13,579.0,26,0,1
4,2022-09-30T23:30:00+02:00,Auvergne-Rhône-Alpes,84,PRO5,P1: ]0-3] kVA,55645,18921676.0,428.0,16,239.0,16,334.0,33,0,1


In [8]:
df.shape

(16314999, 15)

In [9]:
# keep only the columns we need
df = df[["Horodate", "Région", "Code région", "Profil", "Plage de puissance souscrite", "Nb points soutirage", "Total énergie soutirée (Wh)"]]

In [10]:
# split horodate
df["year"] = df["Horodate"].str.split("-", expand=True)[0]
df["month"] = df["Horodate"].str.split("-", expand=True)[1]
df["day"] = df["Horodate"].str.split("-", expand=True)[2].str.split("T", expand=True)[0]

df["time"] = pd.to_datetime(df["Horodate"].str.split("-", expand=True)[2].str.split("T", expand=True)[1].str.split("+", expand=True)[0])
df["date"] = pd.to_datetime(df["Horodate"].str.split("T", expand=True)[0])


In [15]:
df["time"] = df["time"].dt.time

In [12]:
df["Catégorie profil"] = np.where(df["Profil"].str.contains("RES"), "RES", df["Profil"])
df["Catégorie profil"] = np.where(df["Profil"].str.contains("PRO"), "PRO", df["Catégorie profil"])
df["Catégorie profil"] = np.where(df["Profil"].str.contains("ENT"), "ENT", df["Catégorie profil"])


In [13]:
df["Total énergie soutirée (MWh)"] = df["Total énergie soutirée (Wh)"] / 1000000

In [16]:
df.head()

,Horodate,Région,Code région,Profil,Plage de puissance souscrite,Nb points soutirage,Total énergie soutirée (Wh),year,month,day,time,date,Catégorie profil,Total énergie soutirée (MWh)
0,2022-09-30T23:30:00+02:00,Auvergne-Rhône-Alpes,84,ENT3 (+ ENT4 + ENT5),P0: Total <= 36 kVA,1376,2095739.0,2022,09,30,23:30:00,2022-09-30,ENT,2.095739
1,2022-09-30T23:30:00+02:00,Auvergne-Rhône-Alpes,84,PRO1 (+ PRO1WE),P4: ]9-12] kVA,52691,12376840.0,2022,09,30,23:30:00,2022-09-30,PRO,12.376840
2,2022-09-30T23:30:00+02:00,Auvergne-Rhône-Alpes,84,PRO1 (+ PRO1WE),P8: ]24-30] kVA,8350,6545703.0,2022,09,30,23:30:00,2022-09-30,PRO,6.545703
3,2022-09-30T23:30:00+02:00,Auvergne-Rhône-Alpes,84,PRO2 (+ PRO2WE + PRO6),P3: ]6-9] kVA,19759,7072016.0,2022,09,30,23:30:00,2022-09-30,PRO,7.072016
4,2022-09-30T23:30:00+02:00,Auvergne-Rhône-Alpes,84,PRO5,P1: ]0-3] kVA,55645,18921676.0,2022,09,30,23:30:00,2022-09-30,PRO,18.921676


In [17]:
# write to csv file
df.to_csv("conso-inf36-region-clean.csv", index=False)

# Découpage en datasets par région 

In [19]:
# create a dataset for each region
for region in df["Région"].unique():
    df_region = df[df["Région"] == region]
    df_region.to_csv("regions_datasets\conso-inf36-region-{}.csv".format(region), index=False)


# Aggrégation par jour 

In [3]:

regions = ["Auvergne-Rhône-Alpes", "Bourgogne-Franche-Comté", "Bretagne", "Centre-Val de Loire", "Grand-Est", "Hauts-de-France", "Île-de-France", "Normandie", "Nouvelle Aquitaine", "Occitanie", "Pays de la Loire", "Provence-Alpes-Côte d'Azur"]

In [2]:
# create a function to aggregate per day and per region
def aggregate_per_day(file_name):
    df = pd.read_csv(file_name)
    df = df.groupby(["date", "Catégorie profil"]).agg({"Total énergie soutirée (MWh)": "sum", "Nb points soutirage" : "sum"}).reset_index()
    # write to csv file
    df.to_csv("regions_datasets_agg\conso-inf36-region-{}-agg.csv".format(region), index=False)
    return df

In [4]:
for region in regions:
    aggregate_per_day("regions_datasets\conso-inf36-region-{}.csv".format(region))

In [2]:
df_clean = pd.read_csv("conso-inf36-region-clean.csv")

In [3]:
df_clean.head()

,Horodate,Région,Code région,Profil,Plage de puissance souscrite,Nb points soutirage,Total énergie soutirée (Wh),year,month,day,time,date,Catégorie profil,Total énergie soutirée (MWh)
0,2022-09-30T23:30:00+02:00,Auvergne-Rhône-Alpes,84,ENT3 (+ ENT4 + ENT5),P0: Total <= 36 kVA,1376,2095739.0,2022,9,30,23:30:00,2022-09-30,ENT,2.095739
1,2022-09-30T23:30:00+02:00,Auvergne-Rhône-Alpes,84,PRO1 (+ PRO1WE),P4: ]9-12] kVA,52691,12376840.0,2022,9,30,23:30:00,2022-09-30,PRO,12.376840
2,2022-09-30T23:30:00+02:00,Auvergne-Rhône-Alpes,84,PRO1 (+ PRO1WE),P8: ]24-30] kVA,8350,6545703.0,2022,9,30,23:30:00,2022-09-30,PRO,6.545703
3,2022-09-30T23:30:00+02:00,Auvergne-Rhône-Alpes,84,PRO2 (+ PRO2WE + PRO6),P3: ]6-9] kVA,19759,7072016.0,2022,9,30,23:30:00,2022-09-30,PRO,7.072016
4,2022-09-30T23:30:00+02:00,Auvergne-Rhône-Alpes,84,PRO5,P1: ]0-3] kVA,55645,18921676.0,2022,9,30,23:30:00,2022-09-30,PRO,18.921676


In [5]:
# aggregate per day for all regions

df_clean_agg_day = df_clean.groupby(["date", "Catégorie profil"]).agg({"Total énergie soutirée (MWh)": "sum", "Nb points soutirage" : "sum"}).reset_index()
# write to csv file
df_clean_agg_day.to_csv("conso-inf36-region-agg.csv", index=False)


In [4]:
# aggreate per month
df_clean_agg_month = df_clean.groupby(["year", "month", "Catégorie profil"]).agg({"Total énergie soutirée (MWh)": "sum", "Nb points soutirage" : "sum"}).reset_index()
# write to csv file
df_clean_agg_month.to_csv("conso-inf36-region-agg-month.csv", index=False)

In [8]:
# show the types of colunms
df_clean.dtypes

Horodate                         object
Région                           object
Code région                       int64
Profil                           object
Plage de puissance souscrite     object
Nb points soutirage               int64
Total énergie soutirée (Wh)     float64
year                              int64
month                             int64
day                               int64
time                             object
date                             object
Catégorie profil                 object
Total énergie soutirée (MWh)    float64
dtype: object